In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import chardet
with open("companies.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(50000))

# check what the character encoding might be
print(result)

FileNotFoundError: [Errno 2] No such file or directory: 'companies.csv'

In [ ]:
#importing datasets
rounds2 = pd.read_csv('rounds2.csv', encoding='palmos')

In [ ]:
companies = pd.read_csv('companies.csv', encoding = 'palmos')

In [ ]:
companies.head()

In [ ]:
rounds2.head()

In [ ]:
companies.shape

In [ ]:
rounds2.shape

## Checkpoint 1: Data Cleaning 1

In [ ]:
# How many unique companies are present in rounds2?
rounds2['company_permalink'].nunique()

In [ ]:
# How many unique companies are present in companies?
companies['permalink'].nunique()

In [ ]:
# Converting companies unique key to lower case in both dataframes for comparison

rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()
companies['permalink'] = companies['permalink'].str.lower()

In [ ]:
rounds2['company_permalink'].nunique()

In [ ]:
companies['permalink'].nunique()

In [ ]:
# Create list of companies for both dataframe.
rounds2_list = rounds2['company_permalink'].unique().tolist()
companies_list = companies['permalink'].unique().tolist()

# list of companies in **rounds2** which are not present in **companies**. Lets call it rounds2_diff_companies
rounds2_diff_companies = set(rounds2_list).difference(set(companies_list))

In [ ]:
#Number of companies present in **rounds2** which are not present in **companies**
len(rounds2_diff_companies)

#### Merging the dataframes **rounds2** and **companies**

In [ ]:
#rename unique key column in 'rounds2' so that it matches with unique key of 'companies'

rounds2.rename(columns = {'company_permalink':'permalink'}, inplace=True)

In [ ]:
rounds2.head()

In [ ]:
#merging 'companies' with 'rounds2' on variable 'permalink'.
#Since 'companies' file has the data telling the identity of comoany, we will use left join with 'rounds2'.
#By doing left join, each row of rounds2 will have details of the company
master_frame = rounds2.merge(companies, how = 'left', on = 'permalink')

In [ ]:
master_frame.head()

In [ ]:
master_frame.shape

In [ ]:
master_frame.isna().sum()

#### 'funding_round_type' has no null values. This will help to clearly identify no. of investment in each type.
#### Since variation in 'raised_amount_usd' is very high for each funding type, it is difficult to impute the value with any representative value.
#### Also the location of the company is a typical value and cannot be imputed with any representative value.

## Checkpoint 2: Funding Type Analysis

In [ ]:
master_frame['funding_round_type'].value_counts()

In [ ]:
master_frame.dtypes

In [ ]:
funding_type = ['venture', 'seed', 'angel', 'private_equity']

In [ ]:
#creating subset of data with the desired funding round type
master_frame_ft = master_frame[master_frame['funding_round_type'].isin(funding_type)]

In [ ]:
sns.boxplot(data = master_frame_ft, x = 'funding_round_type', y = 'raised_amount_usd')
plt.show()

In [ ]:
for fund in funding_type:
    print (fund)
    print (master_frame_ft[master_frame_ft['funding_round_type'] == fund]['raised_amount_usd'].describe())
    print()

In [ ]:
#removing outliers for each funding type above 2 standard deviations from mean
sd_factor = 2

# standard deviation values for each funding type
sd_list = []
for fund in funding_type:

    sd_list.append((fund, master_frame_ft[master_frame_ft['funding_round_type'] == fund]['raised_amount_usd'].std()))

In [ ]:
sd_list

In [ ]:
master_frame_ft_filtered = pd.DataFrame(columns = master_frame_ft.columns.to_list())
for pair in sd_list:
    df = master_frame_ft[(master_frame_ft['funding_round_type'] == pair[0]) & 
                                      (master_frame_ft['raised_amount_usd'] < sd_factor*pair[1])]
    master_frame_ft_filtered = pd.concat([master_frame_ft_filtered, df], ignore_index= True)

In [ ]:
master_frame_ft_filtered.head()

In [ ]:
master_frame_ft_filtered['funding_round_type'].unique()

In [ ]:
#checking the values for the filtered dataset
for fund in funding_type:
    print (fund)
    print (master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == fund]['raised_amount_usd'].describe())
    print()

In [ ]:
for fund in funding_type:
    sns.boxplot(data = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == fund], 
            x = 'funding_round_type', y = 'raised_amount_usd')
    plt.show()

#### Since all funding types have outliers, it will be wise to use median as representative of data for each category.
#### The IQR of funding type 'private_equity' is from 5 million to 70 million. Since our budget of 5 to 15 million USD lies in this range, this investment type is most suitable for the company.

## Checkpoint 3: Country Analysis

In [ ]:
#creating subset of data where funding_round_type is 'private_equity'
master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'private_equity'].isna().sum()

In [ ]:
#filtering data for funding type 'private_equity'
master_frame_ft_filtered_pe = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'private_equity']

In [ ]:
# creating list of top 9 countries for funding type 'private_equity' based on total amount invested
top9 = pd.pivot_table(data = master_frame_ft_filtered_pe, index = 'country_code', values = 'raised_amount_usd', aggfunc = sum)

In [ ]:
top9 = pd.DataFrame(top9).reset_index()

In [ ]:
top9.head()

In [ ]:
top9 = top9.sort_values(by='raised_amount_usd', ascending=False).reset_index(drop=True)

In [ ]:
top9 = top9.head(9)

In [ ]:
top9

In [ ]:
#top 9 countries for funding type 'private_equity'
top9['country_code'].values.tolist()

#### From the list of top 9 countries for funding type 'private_equity', the top 3 english speaking countries are USA, GBR and IND.

## Checkpoint 4: Sector Analysis 1

In [ ]:
# importing mapping file for analysis
mapping = pd.read_csv('mapping.csv')

In [ ]:
mapping.head()

In [ ]:
mapping.dtypes

In [ ]:
mapping['category_list'].value_counts()

In [ ]:
mapping.shape

In [ ]:
mapping[mapping['Blanks'] == 1]

In [ ]:
# Since sector Blank has only one category_list wih null value, we can delete this column and the first row
mapping.drop('Blanks', axis = 1, inplace=True)
mapping.dropna(subset=['category_list'], inplace=True)

In [ ]:
mapping.reset_index(drop=True, inplace=True)

In [ ]:
#checking for any null values
mapping.isna().sum()

In [ ]:
#creating primary sector from category_list column of master_frame data

master_frame_ft_filtered_pe['primary_sector'] = master_frame_ft_filtered_pe['category_list'].apply(lambda x: str(x).split('|')[0])

In [ ]:
master_frame_ft_filtered_pe.head()

In [ ]:
#merging dataset with mapping file
master_frame_ft_filtered_pe = master_frame_ft_filtered_pe.merge(mapping, how= 'inner', left_on='primary_sector', right_on='category_list')

In [ ]:
master_frame_ft_filtered_pe.head()

In [ ]:
master_frame_ft_filtered_pe.drop('category_list_y', axis = 1, inplace = True)

In [ ]:
master_frame_ft_filtered_pe.rename(columns={'category_list_x':'category_list'}, inplace=True)

In [ ]:
master_frame_ft_filtered_pe.head()

## Checkpoint 5: Sector Analysis 2

#### Top 3 english speaking countries are USA, GBR and IND
#### Creating dataframe for each countries with funding type as 'private_equity', where investment range is 5 to 15 million USD
#### D1 --> USA, D2 --> GBR, and D3 --> IND

In [ ]:
#creating dataframe D1 for USA
D1 = master_frame_ft_filtered_pe[(master_frame_ft_filtered_pe['country_code'] == 'USA') 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] >= 5000000) 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] <= 15000000)]

#creating dataframe D2 for GBR
D2 = master_frame_ft_filtered_pe[(master_frame_ft_filtered_pe['country_code'] == 'GBR') 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] >= 5000000) 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] <= 15000000)]

#creating dataframe D3 for IND
D3 = master_frame_ft_filtered_pe[(master_frame_ft_filtered_pe['country_code'] == 'IND') 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] >= 5000000) 
                                 & (master_frame_ft_filtered_pe['raised_amount_usd'] <= 15000000)]

In [ ]:
# Total number of investment in USA
D1.shape

In [ ]:
# Total number of investment in GBR
D2.shape

In [ ]:
# Total number of investment in IND
D3.shape

In [ ]:
# Total amount of investment in all 3 countries
print(D1['raised_amount_usd'].sum())
print(D2['raised_amount_usd'].sum())
print(D3['raised_amount_usd'].sum())

In [ ]:
# List of 8 main sectors
sector_list = mapping.columns.to_list()[1:]

In [ ]:
sector_list

In [ ]:
# Top sectors in USA where most number of investment where done.
D1[sector_list].sum()

In [ ]:
# Top sectors in GBR where most number of investment where done.
D2[sector_list].sum()

In [ ]:
D2[sector_list + ['raised_amount_usd']].sort_values(by='raised_amount_usd', ascending = False)

In [ ]:
# Top sectors in IND where most number of investment where done.
D3[sector_list].sum()

In [ ]:
D3[sector_list + ['raised_amount_usd']].sort_values(by='raised_amount_usd', ascending = False)

#### In USA, the highest number of investments were for sectors: Cleantech / Semiconductors = 99, Others = 61 and, Health = 34

#### In GBR, the highest number of investments were for sectors: Social, Finance, Analytics, Advertising = 2, Manufacturing = 2 and, Others = 2

#### In IND, the highest number of investments were for sectors: Entertainment = 1, Others = 1 and, Cleantech / Semiconductors = 1

In [ ]:
#company recieving highest funding in Cleantech / Semiconductors sector which are under operation in USA
D1[(D1['Cleantech / Semiconductors'] == 1) & (D1['status'] == 'operating')].sort_values(by=('raised_amount_usd'), ascending=False)

In [ ]:
#company recieving highest funding in software sector which are under operation in USA
D1[(D1['Others'] == 1) & (D1['status'] == 'operating')].sort_values(by=('raised_amount_usd'), ascending=False)

In [ ]:
#company recieving highest funding in consulting sector in GBR
D2[D2['Social, Finance, Analytics, Advertising'] == 1].sort_values(by=('raised_amount_usd'), ascending=False)

In [ ]:
#company recieving highest funding in manufacturing sector in GBR
D2[D2['Manufacturing'] == 1].sort_values(by=('raised_amount_usd'), ascending=False)

In [ ]:
#company recieving highest funding in Jewelry sector in IND
D3[D3['Entertainment'] == 1].sort_values(by=('raised_amount_usd'), ascending=False)

In [ ]:
#company recieving highest funding in Hospitality sector in IND
D3[D3['Cleantech / Semiconductors'] == 1].sort_values(by=('raised_amount_usd'), ascending=False)

## Checkpoint 6: Plots

### Plot 1

In [ ]:
#Creating dataset for each funding type

venture_data = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'venture']['raised_amount_usd']
seed_data = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'seed']['raised_amount_usd']
angel_data = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'angel']['raised_amount_usd']
pe_data = master_frame_ft_filtered[master_frame_ft_filtered['funding_round_type'] == 'private_equity']['raised_amount_usd']

In [ ]:
dataset = [venture_data, seed_data, angel_data, pe_data]
dataset_label = ['venture', 'seed', 'angel', 'private_equity']

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
ax.boxplot(dataset, labels = dataset_label)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
plt.show()

In [ ]:
#creating distribution plot for each funding type.

plt.figure(figsize=(20,15), dpi = 120)
funding_range = [5000000,1500000]
for i,j in enumerate(dataset):
    mean = dataset[i].mean()
    median = dataset[i].median()
    value_25_percentile = np.percentile(dataset[i], 25)
    value_75_percentile = np.percentile(dataset[i], 75)
    
    ax = plt.subplot(2,2,i+1)
    sns.kdeplot(data = dataset[i], shade = True)
    sns.scatterplot([median],[0], color='red', label='median')
    sns.scatterplot([mean],[0], color='blue', label='mean')
    sns.scatterplot([value_25_percentile], [0], label = '25th percentile', color = 'green')
    sns.scatterplot([value_75_percentile], [0], label = '75th percentile', color = 'orange')
    sns.lineplot(funding_range, [0,0], color='black', label = 'Range', markers=True, linewidth = 5)
    plt.title('mean = {}, median = {},\n 25th_percentile = {},\n 75th_percentile = {}'.format(round(mean),median,value_25_percentile
                                                                                     ,value_75_percentile))
    plt.xlabel(dataset_label[i])
    plt.ylabel('Density')
    ax.spines['top'].set_visible(False)
    #ax.spines['right'].set_visible(False)
    #ax.spines['left'].set_visible(False)
    #ax.spines['bottom'].set_visible(False)
    plt.subplots_adjust(left=0.1,
                    bottom=0.1,  
                    top=0.9, 
                    wspace=0.4, 
                    hspace=0.4)
plt.savefig('Density Plot')
plt.show()

#### If we check our funding range, we can see that in private_equity funding it is in the IQR range. This makes it a preferred investment option for us.


### Plot 2

In [ ]:
sns.barplot(data=top9.iloc[:], x='country_code', y='raised_amount_usd')
plt.savefig('top9_1')
plt.show()

In [ ]:
#barplot excluding USA
sns.barplot(data=top9.iloc[1:], x='country_code', y='raised_amount_usd')
plt.savefig('top9_2')
plt.show()

#### In top 9 countries against the total amount of investments of funding type 'private_equity', USA has highest investement. It is followed by CHN, GBR and IND.
#### Since we are interested in only english speaking countries, we will omit CHN. Now, top 3 countries of our interest are USA, GBR and IND.

### Plot 3

### Number of investments in the top 3 sectors of the top 3 countries

In [ ]:

USA_list = pd.Series(D1[sector_list].sum()).reset_index()
GBR_list = pd.Series(D2[sector_list].sum()).reset_index()
IND_list = pd.Series(D3[sector_list].sum()).reset_index()

In [ ]:
investment_list = [USA_list, GBR_list, IND_list]
label = ['USA_investment', 'GBR_investment', 'IND_investment']
plt.figure(figsize=(20,12), dpi = 200)
for i,j in enumerate(investment_list):
    ax = plt.subplot(3,1,i+1)
    sns.barplot(data = investment_list[i], x = 'index', y = 0, estimator=sum)
    ax.bar_label(ax.containers[0])
    ax.set_xlabel('')
    ax.set_ylabel('Number of Investments')
    ax.set_title(label=label[i])
plt.savefig('investment count')